In [1]:
import pandas as pd
from filterpy.kalman import KalmanFilter
from filterpy.common import Q_discrete_white_noise
import numpy as np
import matplotlib.pyplot as plt 
import plotly.graph_objects as go
from plotly_resampler import FigureResampler, FigureWidgetResampler

In [2]:
df = pd.read_parquet("../data/static_models/pressure_miso.par")
df = df.set_index("time")
df = df.rolling(window=60, center=True).mean().dropna()
df = df.resample("1min").mean().dropna()

In [3]:
train_df = df["2024-01-09 17:00:00":"2024-01-10 00:30:00"]
test_df = df["2024-01-11":"2024-02-16 12:00:00"]

In [4]:
from src.system_identification import ARXParametersIdentification
from src.preprocessing import Preprocessor
import pandas as pd
from sysidentpy.basis_function._basis_function import Polynomial

In [5]:
FEATURES = "outflow"
TARGET = 'pressure'

sys_id = ARXParametersIdentification(FEATURES, 
                            TARGET, 
                            dataframe= train_df, 
                            model_type= 'siso')

In [6]:
parameters = {
    'estimator': 'ridge_regression',
    'ridge_param': 0.5,
    'order_selection': False,
    'n_terms': 3,
    'extended_least_squares': False,
    'ylag': 2,  
    'xlag': 1, 
    'basis_function': Polynomial(degree=1)
}
    
fitted_model, model_parameters = sys_id.identify_arx_parameters(x_train=train_df[FEATURES], 
                                                                y_train= train_df[TARGET],
                                                                **parameters)
    
model_parameters

,Regressors,Parameters,ERR
0,1,3.9617E-01,9.9852E-01
1,x1(k-1),1.9830E-04,8.2200E-05
2,y(k-2),1.6274E-01,2.6041E-06


In [7]:
yhat  = sys_id.predict(x_test= test_df[FEATURES], y_test= test_df[TARGET], n_steps_ahead= 200)

#val_plots = sys_id.residuals_analysis(yhat, df[TARGET].values)
#print(model_parameters)

RRSE: 0.6233215319258801
MSE: 0.0005967271988107548


In [8]:
test_df["yhat"] = yhat

sys_id.plot_features_and_target(test_df, ['pressure', 'yhat'])

FigureWidgetResampler({
    'data': [{'name': '<b style="color:sandybrown">[R]</b> pressure <i style="color:#fc9944">~53m</i>',
              'showlegend': True,
              'type': 'scattergl',
              'uid': '1d0dd355-2bf2-4993-adda-670f06d60984',
              'x': array([datetime.datetime(2024, 1, 11, 0, 0, tzinfo=<UTC>),
                          datetime.datetime(2024, 1, 11, 0, 28, tzinfo=<UTC>),
                          datetime.datetime(2024, 1, 11, 0, 58, tzinfo=<UTC>), ...,
                          datetime.datetime(2024, 2, 16, 10, 8, tzinfo=<UTC>),
                          datetime.datetime(2024, 2, 16, 11, 2, tzinfo=<UTC>),
                          datetime.datetime(2024, 2, 16, 11, 26, tzinfo=<UTC>)], dtype=object),
              'y': array([0.60422277, 0.58209249, 0.60835444, ..., 0.6283195 , 0.63881493,
                          0.63650554])},
             {'name': '<b style="color:sandybrown">[R]</b> yhat <i style="color:#fc9944">~53m</i>',
              '